In [242]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn import ensemble

warnings.filterwarnings('ignore')

# LIMPIEZA

In [231]:
data = pd.read_csv('diamonds_train.csv')
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.78,Premium,F,VS1,61.5,58.0,5.93,5.98,3.66,3446
1,1,0.31,Ideal,D,SI1,60.8,56.0,4.37,4.32,2.64,732
2,2,0.30,Ideal,F,SI1,62.3,54.0,4.30,4.34,2.69,475
3,3,1.04,Ideal,E,VVS2,62.0,58.0,6.54,6.46,4.03,9552
4,4,0.65,Ideal,J,SI1,61.4,55.0,5.58,5.62,3.44,1276


In [232]:
# Nos quitamos el id
data = data.iloc[:,1:]
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.78,Premium,F,VS1,61.5,58.0,5.93,5.98,3.66,3446
1,0.31,Ideal,D,SI1,60.8,56.0,4.37,4.32,2.64,732
2,0.30,Ideal,F,SI1,62.3,54.0,4.30,4.34,2.69,475
3,1.04,Ideal,E,VVS2,62.0,58.0,6.54,6.46,4.03,9552
4,0.65,Ideal,J,SI1,61.4,55.0,5.58,5.62,3.44,1276


In [233]:
''' Vemos que en las columnas de medidas (x,y,z) los valores mínimos son 0, por tanto, son valores nulos que nos
desvirtúan las métricas.'''
'''Además, en la medida y el valor máximo es 58.9, que comparado con la media parece traterse de un error'''
data.describe()

,carat,depth,table,x,y,z,price
count,40345.000000,40345.000000,40345.000000,40345.000000,40345.000000,40345.000000,40345.000000
mean,0.797699,61.749721,57.449421,5.730824,5.734808,3.537958,3931.902243
std,0.474309,1.427728,2.234087,1.121767,1.151536,0.695039,3994.318832
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,326.000000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,952.000000
50%,0.700000,61.800000,57.000000,5.690000,5.710000,3.520000,2397.000000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.030000,5317.000000
max,5.010000,79.000000,95.000000,10.740000,58.900000,8.060000,18823.000000


In [234]:
# Teniendo en cuenta lo anterior, quitamos los valores nulos
data = data.loc[(data['x'] != 0) & (data['y'] != 0) & (data['z'] != 0)]
data = data.loc[(data['y'] < 15)]

In [235]:
# Sacamos lista ordenada del tipo de corte del peor al mejor
cut_list = list(data.cut.value_counts().keys())
cut_list.reverse()
# Aplicamos el valor numérico para que lo valore en función de la importancia del corte
data['cut'] = data['cut'].apply(lambda x: cut_list.index(x))

In [236]:
# Sacamos lista ordenada del color del peor al mejor
color_list = sorted(list(data.color.value_counts().keys()),reverse=True)
# Aplicamos el valor numérico para que lo valore en función de la importancia del color
data['color'] = data['color'].apply(lambda x: color_list.index(x))

In [237]:
# Sacamos lista ordenada de la claridad de peor a mejor
clarity_list = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
# Aplicamos el valor numérico para que lo valore en función de la importancia de la claridad
data['clarity'] = data['clarity'].apply(lambda x: clarity_list.index(x))

In [238]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.78,3,4,4,61.5,58.0,5.93,5.98,3.66,3446
1,0.31,4,6,2,60.8,56.0,4.37,4.32,2.64,732
2,0.30,4,4,2,62.3,54.0,4.30,4.34,2.69,475
3,1.04,4,5,5,62.0,58.0,6.54,6.46,4.03,9552
4,0.65,4,0,2,61.4,55.0,5.58,5.62,3.44,1276


In [212]:
# Hago copia de los datos para que al normalizarlos y cambiar todos los valores, no me varíen las métricas del describe()
data_stable = data.copy()

In [213]:
# Convierto datos en array
data_array = data.to_numpy().T

In [155]:
# Pruebas para ver si funciona lo que tengo en mente
x = np.array([[ 0.,  1.,  2.],[ 3.,  4.,  5.],[ 6.,  7.,  8.]])
x.shape
x = x.T
print(x)
a = x[0]*2
b = x[1]*4
x = a,b
pd.DataFrame(x)
x

[[0. 3. 6.]
 [1. 4. 7.]
 [2. 5. 8.]]


(array([ 0.,  6., 12.]), array([ 4., 16., 28.]))

In [106]:
# Función para normalizar, la cual no funciona
def normalizer(array):
    count = 0
    for col in list(data.columns):
        ((array[count]-data_stable.describe()[col][1])/data_stable.describe()[col][2])
        count += 1
        print(f'{col} done')
    return array

In [215]:
# Por lo que hice este churro por no rallarme más con la función anterior
a = (data_array[0]-data_stable.describe()['carat'][1])/data_stable.describe()['carat'][2]
b = (data_array[1]-data_stable.describe()['cut'][1])/data_stable.describe()['cut'][2]
c = (data_array[2]-data_stable.describe()['color'][1])/data_stable.describe()['color'][2]
d = (data_array[3]-data_stable.describe()['clarity'][1])/data_stable.describe()['clarity'][2]
e = (data_array[4]-data_stable.describe()['depth'][1])/data_stable.describe()['depth'][2]
f = (data_array[5]-data_stable.describe()['table'][1])/data_stable.describe()['table'][2]
g = (data_array[6]-data_stable.describe()['x'][1])/data_stable.describe()['x'][2]
h = (data_array[7]-data_stable.describe()['y'][1])/data_stable.describe()['y'][2]
i = (data_array[8]-data_stable.describe()['z'][1])/data_stable.describe()['z'][2]
j = (data_array[9]-data_stable.describe()['price'][1])/data_stable.describe()['price'][2]

In [216]:
normalized_data = a,b,c,d,e,f,g,h,i,j
normalized_data = pd.DataFrame(normalized_data).T

In [217]:
normalized_data.columns = list(data.columns)
normalized_data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,-0.036794,0.084257,0.348828,0.571595,-0.175145,0.246645,0.177732,0.222286,0.174898,-0.121282
1,-1.028201,0.980822,1.524677,-0.639876,-0.665569,-0.648787,-1.215041,-1.270667,-1.299874,-0.801059
2,-1.049295,0.980822,0.348828,-0.639876,0.385339,-1.544220,-1.277537,-1.252680,-1.227581,-0.865430
3,0.511643,0.980822,0.936752,1.177331,0.175157,0.246645,0.722342,0.653983,0.709864,1.408090
4,-0.311013,0.980822,-2.002872,-0.639876,-0.245206,-1.096503,-0.134749,-0.101487,-0.143190,-0.664803


In [239]:
# Asignamos las features a la X
X = data.drop(columns='price')
# Asignamos el precio a predecir a la y
y = data['price']

In [240]:
# Dividimos los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [187]:
# Entrenamos según modelo

models = {
    'LinearReg': LinearRegression(),
    'LinearSVR': LinearSVR(),
    'RandomForestRegressor': RandomForestRegressor(n_estimators=250),
    'XGBRegressor': xgb.XGBRegressor(n_estimators=250),
    'HistGrad': HistGradientBoostingRegressor(),
    'ElasticNet': ElasticNet(random_state=0)
    }

results = {}

for name,model in models.items():
    
    data_trained = model.fit(X_train, y_train)
    prediction = data_trained.predict(X_test)
    desviation = 1-np.abs(y_test-prediction).sum()/len(y_test)
    rmse = sqrt(mean_squared_error(y_test, prediction))
    
    results[name] = desviation, rmse

In [188]:
results

{'ElasticNet': (0.5083882857028028, 0.6847950035452086),
 'HistGrad': (0.9281390998045487, 0.1369333995007283),
 'LinearReg': (0.8028812973179986, 0.29373063478989686),
 'LinearSVR': (0.8273322874980513, 0.3233159934062042),
 'RandomForestRegressor': (0.9304523696973638, 0.14053537871100988),
 'XGBRegressor': (0.9284979281965247, 0.1411318641078588)}

In [243]:
# Entrenamos el modelo
model = ensemble.GradientBoostingRegressor(n_estimators=400,max_depth=5,min_samples_split=2,learning_rate=0.1,loss='ls')
data_trained = model.fit(X_train, y_train)

In [244]:
# Lo aplicamos para hacer la predicción
pred = data_trained.predict(X_test)

In [245]:
pred

array([5926.02995715, 4385.27457292, 2052.76576733, ..., 4149.43286838,
        871.34138671,  648.42492874])

In [224]:
type(y_test)
pred.shape

(8066,)

In [225]:
y_test = y_test.to_numpy()
y_test.shape

(8066,)

In [226]:
# Desnormalizo los resultados
y_test = (y_test*data_stable.describe()['price'][2]+data_stable.describe()['price'][1])
pred = (pred*data_stable.describe()['price'][2]+data_stable.describe()['price'][1])

In [247]:
# La diferencia media que hay entre la realidad y la predicción
1-np.abs(y_test-pred).sum()/len(y_test)

-269.60410678260973

In [248]:
# Cuanto de similares de media son los datos reales vs predichos
rms = sqrt(mean_squared_error(y_test, pred))
rms

530.2689133791034